In [1]:
import numpy as np
import pandas as pd
import requests #HTTP request
import xlsxwriter
import math

#Ignores all the warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reads the .csv files for each of the tickers.
stocks = pd.read_csv('sp_500_stocks.csv', encoding='utf-8')

In [3]:
#API Token to retrieve data through the API.
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [4]:
#symbol='AAPL'
#api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(api_url).json()

In [5]:
#Columns for the outputed data.
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
#final_dataframe = pd.DataFrame(columns = my_columns)

In [6]:
print("Please enter the minimum market cap you would like for your portfolio:\n(The lowest market cap is 1,500,000,000)")
min = int(input())
print("Please enter the maximum market cap you would like for your portfolio:\n(The highest market cap of a Fortune 500 Company is 2,420,000,000,000)")
max = int(input())

Please enter the minimum market cap you would like for your portfolio:
(The lowest market cap is 1,500,000,000)
1500000000
Please enter the maximum market cap you would like for your portfolio:
(The highest market cap of a Fortune 500 Company is 2,420,000,000,000)
2420000000000


In [7]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

#Split symbols into chunks of 100 stock symbols.
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    #Creates a list of all the chunks, turning them into strings connected by commas.
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    #Performs API call for each batch, to save time from doing it one by one.
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    #Adds the symbol's gotten values separately.
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['marketCap'],
                                                   'N/A'],
                                                  index = my_columns),
                                        ignore_index = True)

In [8]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:100000


In [9]:
if len(final_dataframe.index != 0):
    position_size = float(portfolio_size) / len(final_dataframe.index)
    for i in range(0, len(final_dataframe['Ticker'])):
        final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,130.64,38740003969,1
1,AAL,13.86,8990711204,14
2,AAP,175.00,10271332945,1
3,AAPL,157.40,2473967563281,1
4,ABBV,147.16,264994413501,1
...,...,...,...,...
494,YUM,116.37,33183045725,1
495,ZBH,114.57,23816719598,1
496,ZBRA,291.15,15441917247,0
497,ZION,59.80,9010486303,3


In [10]:
#Start of Excel Export
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [11]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [12]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [13]:
writer.save()